In [1]:
import numpy as np
import pandas as pd

In [3]:
from preprocessing import str2coords
def load_data(PATH='../../data/pku-autonomous-driving/'):
    train = pd.read_csv(PATH + 'train.csv')
    test = pd.read_csv(PATH + 'sample_submission.csv')
    return train, test

In [4]:
train,test = load_data('../data/pku-autonomous-driving/')

In [5]:
res1 = str2coords(train.values[0][1])

In [6]:
yaws = np.array([item['yaw'] for item in res1])
pitchs = np.array([item['pitch'] for item in res1])
rolls = np.array([item['roll'] for item in res1])


In [2]:
def euler_to_quaternion(yaw, pitch, roll):
    qx = np.sin(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) - np.cos(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
    qy = np.cos(roll/2) * np.sin(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.cos(pitch/2) * np.sin(yaw/2)
    qz = np.cos(roll/2) * np.cos(pitch/2) * np.sin(yaw/2) - np.sin(roll/2) * np.sin(pitch/2) * np.cos(yaw/2)
    qw = np.cos(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)

    return [qx, qy, qz, qw]

In [7]:
qx,qy,qz,qw = euler_to_quaternion(yaws, pitchs,rolls)

In [8]:
def normalized(qx,qy,qz,qw):
    qd = np.sqrt(qx**2+qy**2+qz**2+qw**2)
    return qx/qd,qy/qd,qz/qd,qw/qd

In [12]:
def inverse(qx,qy,qz,qw):
    qd2 = qx**2+qy**2+qz**2+qw**2
    return -qx/qd2, -qy/qd2, -qz/qd2, qw/qd2

In [22]:
def multiply_w(qx1,qy1,qz1,qw1,qx2,qy2,qz2,qw2):
    return qw2*qw1 - qx1*qx2-qy1*qy2-qz1*qz2

In [25]:
def RotationDistance(item1, item2):
    #item: (yaw,pitch,roll)
    
    qx1,qy1,qz1,qw1 = normalized(*euler_to_quaternion(*item1))
    qx2,qy2,qz2,qw2 = normalized(*euler_to_quaternion(*item2))
    
    diff_w = multiply_w(qx1,qy1,qz1,qw1, 
                      *inverse(qx2,qy2,qz2,qw2))
    diff_w = max(diff_w, -1.)
    diff_w = min(1, diff_w)
    return np.degrees(np.arccos(diff_w))

In [28]:
def TranslationDistance(o1, o2):
    #o: (x,y,z)
    o1 = np.array(o1)
    o2 = np.array(o2)
    return np.sqrt(np.sum((o1-o2)**2))

In [ ]:
def obtain_score(s_pred, s_true):
    label1 = str2coords(s_pred)
    label2 = str2coords(s_true)
    
    yaws1 = np.array([item['yaw'] for item in label1])
    pitchs1 = np.array([item['pitch'] for item in label1])
    rolls1 = np.array([item['roll'] for item in label1])
    x1 = np.array([item['x'] for item in label1])
    y1 = np.array([item['y'] for item in label1])
    z1 = np.array([item['z'] for item in label1])
    
    yaws2 = np.array([item['yaw'] for item in label2])
    pitchs2 = np.array([item['pitch'] for item in label2])
    rolls2 = np.array([item['roll'] for item in label2])
    x2 = np.array([item['x'] for item in label2])
    y2 = np.array([item['y'] for item in label2])
    z2 = np.array([item['z'] for item in label2])
    
    for i,(yaw_p,pitch_p,roll_p,x_p,y_p,z_p) in enumerate(zip(yaws1,
                                      pitchs1,
                                      rolls1,
                                      x1,
                                      y1,
                                      z1)):
        for j,(yaw_t,pitch_t,roll_t,x_t,y_t,z_t) in enumerate(zip(yaws2,
                                      pitchs2,
                                      rolls2,
                                      x2,
                                      y2,
                                      z2)):
            dist1 = RotationDistance((yaw_p,pitch_p,roll_p),
                (yaw_t,pitch_t,roll_t))
            
            dist2 = TranslationDistance((x_p,y_p,z_p),
                                       (x_t,y_t,z_t))
            
    